# Validation d'un modèle de régression

Lorsque nous avons une modèle de régression linéaire simple, l'évaluation de la qualité du modèle est relativement simple. Cependant, plus un modèle de régression est sophistiqué, plus il se pose la question de comment valider le modèle.

Une régression est au final un modèle de prédiction ou d'estimation pour la variable dépendante sur la base des variables indépendantes. La meilleure façon d'évaluer la qualité du modèle est d'**évaluer sa capacité de faire des prédictions correctes sur la base de nouvelles données**. Ceci permet également d'éviter qu'un modèle puisse se baser sur le fait d'avoir déjà «vu» une donnée particulière et de l'utiliser pour la prédiction. Ceci porte le danger de faire une prédiction parfaite pour toutes les données connues, et de faire une mauvaise prédiction dans tous les autres cas. On dit alors que le modèle fait du **over-fitting** et qu'il **généralise mal** aux nouvelles données. C'est un problème avec tous les modèles de régression sophistiqués.

La stratégie générale que nous pouvons adapter est la suivante: le jeu de données que nous avons au début est divisé en deux jeux:

1. un **jeu de données de calibration** pour déterminer les paramètres du modèle
2. un **jeu de données de test** pour déterminer la qualité du modèle au niveau de la capacité de prédiction

Nous allons typiquement utiliser 80-90% du jeu de données initial pour la calibration du modèle, et le 10-20% restant pour la validation.

Regardons comment on peut diviser un jeu de données initial en jeux de données de calibration et de test. Commençons par lire un fichier de données:

In [1]:
d = read.csv(file="ch-socioeco-typologie.tsv", sep="\t")

Dans ce cas, nous utilisons 90% des données pour le jeu de données de calibration:

In [3]:
idx = sample(nrow(d), size=0.9*nrow(d))
dtrain = d[idx,]
dtest = d[-idx,]

Les 90% sont spécifiés avec le `0.9` dans le paramètre `size`. Il faut évidemment encore ajuster le nom de la variable `d`.

Pour pouvoir illustrer l'évaluation d'un modèle, nous devons d'abord en faire un. Prenons le [modèle de régression logistique de la semaine passée](../25-regression/regression-logistique.ipynb):

In [5]:
reglogit = glm(typologie_binaire ~ ADO + NADHO + NADRET + AD3PRIM + AD3SEC, family=binomial(link="logit"), data=dtrain) 

Notez que nous avons utilisé le jeu de données __`dtrain`__ et non `d`. 

Maintenant nous pouvons appliquer le modèle au jeu de données de test (`newdata=dtest`):

In [18]:
reglogit_predictions = predict(reglogit, newdata=dtest)

Dans le cas d'une régression logistique, nous pouvons faire un **tableau simple avec le nombre de cas prédits correctement ou non**.

Avec notre régression logistique binomiale, nous devons d'abord traduire la probabilité estimée en classes (`U` pour les centres urbains, `N` pour les autres), ce que nous pouvons faire avec une condition:

In [22]:
pred_classes = ifelse(reglogit_predictions > 0.5, 'U', 'N')

Et maintenant nous pouvons dresser notre tableau de comparaison:

In [23]:
table(pred_classes, dtest$typologie_binaire)

            
pred_classes  N  U
           N 40 13
           U 13 71

Notez que ce tableau est très similaire que nous avons déjà calculé la semaine passée, sauf que maintenant nous l'avons calculé sur le jeu de données de test.

Et nous pouvons calculer la proportion de réponses justes:

In [24]:
sum(pred_classes == dtest$typologie_binaire) / nrow(dtest)

[1] 0.810219

ce qui nous donne 81% de réponses justes pour le jeu de données de test.

Nous pouvons évidemment faire le même calcul pour le jeu de données de calibration:

In [25]:
pred_train = predict(reglogit, newdata=dtrain)
pred_train_classes = ifelse(pred_train > 0.5, 'U', 'N')
sum(pred_train_classes == dtrain$typologie_binaire) / nrow(dtrain)

[1] 0.8558632

Dans la grande majorité des cas, la proportion de réponses justes est plus élevée pour le jeu de données de calibration. Ceci est normal étant donné que le modèle a pu être ajusté spécifiquement à ce jeu de données de calibrage. Par contre, c'est uniquement la proportion de réponses justes pour le jeu de données de test qui est véritablement importante.

Pour les cas où la **variable dépendante est continue** (et non catégorielle), nous pouvons par exemple calculer l'erreur absolu moyen (Mean Absolute Error MAE) sur la base du jeu de données de test. Voici l'exemple de la régression linéaire de la semaine passée:

In [30]:
d2 = read.csv(file="../25-regression/data-zh-be.tsv", sep="\t")
idx = sample(nrow(d2), size=0.9*nrow(d2))
dtrain2 = d2[idx,]
dtest2 = d2[-idx,]
reglin = lm(PMSDIV ~ ADCFARM + PRPROT + PRCATH + PRJEW + PFGEN + PFBAC, data=dtrain2)
lm_pred = predict(reglin, newdata=dtest2)


`lm_pred` contient maintenant les valeurs prédites pour le jeu de données test. Nous pouvons calculer l'erreur absolu moyen avec:

In [32]:
summary(abs(lm_pred - dtest2$PMSDIV))

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  6.046  26.430  43.800  63.760  75.760 239.100 

Nous avons donc un erreur moyen d'environ 64 personnes pour le nombre de personnes divorcées. L'erreur absolu médian quant à lui est d'environ 44 personnes.

Est-ce que c'est beaucoup ou peu? Nous pouvons mettre en relation cette valeur avec le nombre de personnes divorcées:

In [33]:
erreur_relative = abs(lm_pred - dtest2$PMSDIV) / dtest2$PMSDIV

In [34]:
summary(erreur_relative)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.01356 0.05250 0.09845 0.13540 0.17640 0.72430 

ce qui nous fait donc un erreur absolu moyen relatif d'environ 13.5%, et l'erreur absolu médian relatif d'environ 9.8%. 

## Cross-validation (validation croisée)

La méthode de validation présentée ci-dessus est très simple et facile à comprendre. Par contre, elle présente quelques problèmes:

- en cas de petits jeux de données, réserver une bonne partie des données uniquement pour les tests peut être problématique

- en raison de la sélection aléatoire du jeu de données de calibration et de test, chaque calcul du modèle donnera une erreur légèrement différente

En conséquence, plusieurs méthodes alternatives ont été développées. La cross-validation est une telle alternative, qui est plutôt une famille de méthodes que d'une seule méthode, car il y a beaucoup de variantes qui existent. Nous allons nous limiter à deux cas, la ___«leave-one-out cross-validation»___ et la ___«k-fold cross-validation»___.

Dans le cas de la **«leave-one-out cross-validation»**, on enlève une ligne du jeu de données, puis on calcule le modèle sur les données restante, et on calcule l'erreur d'estimation pour la ligne enlevée. On refait cette procédure pour chaque ligne, et on calcule l'erreur moyen.

Dans le cas de la ___«k-fold cross-validation»___, on divise le jeu de données initial en $k$ partitions de taille égale. Une partition sert de jeu de validation et les autres $k-1$ partitions pour la calibration. Puis on refait la même procédure avec une autre partition pour la validation.

Le principe reste au final le même que celui présenté plus haut.

Nous n'allons pas calculer la validation croisée ici, mais il est une bonne chose de connaître à peu près le principe.